# Spotify Suggestion Model V3

## Infrastructure

In [ ]:
import sys
# !{sys.executable} -m pip install spotipy

In [ ]:
username = "agwx2"
cid = "17c11cf9271f4980a85e6aab83cba6bb" 
secret = "015c9abf126340a7981e7ba64dbe7828"
# good_playlist_id = "61BmnG6d7ifmj8Dr5tuCBK"
# bad_playlist_ids = "53FPslFSIAGvVyERcNG4J3"
# source_playlist_id = "5ghFHvZmV4FDK5YLeZQRu9"
good_playlist_ids = [
    "37i9dQZF1DX48TTZL62Yht",   # Hip Hop Favourites
    "37i9dQZF1EQnqst5TRi17F",   # Hip Hop Mix
    "37i9dQZF1DX7FY5ma9162x",   # R&B Favourites
    "37i9dQZF1EQoqCH7BwIYb7",   # R&B Mix
    "53FPslFSIAGvVyERcNG4J3",   # Trap/R&B (personal mix)
    # "37i9dQZF1DWWEcRhUVtL8n"    # Indie Pop
]
bad_playlist_ids = [
    "37i9dQZF1EQpj7X7UK8OOF",   # Rock Mix
    "37i9dQZF1DX1kCIzMYtzum",   # EDM
    "37i9dQZF1DXbITWG1ZJKYt",   # Jazz Classics
    "37i9dQZF1EQmPV0vrce2QZ",   # Country Mix
    "37i9dQZF1EQncLwOalG3K7",   # Pop Mix
    "61BmnG6d7ifmj8Dr5tuCBK"    # Classic Rock (personal mix)
]
source_playlist_ids = [
    # "37i9dQZF1DXb57FjYWz00c",   # 80s Hits
    # "37i9dQZF1EQn2GRFTFMl2A",   # 90s Mix
    # "37i9dQZF1DWUZv12GM5cFk",   # 2000s Hits
    # "37i9dQZF1DXc6IFF23C9jj",   # 2010s Hits
    "5ghFHvZmV4FDK5YLeZQRu9",   # Top 100 (personal mix)
]
source_playlist_id = source_playlist_ids[0]

In [ ]:
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials

import pandas as pd
from sklearn.metrics import accuracy_score, mean_squared_error, r2_score
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split


from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, StackingClassifier
from sklearn.linear_model import LinearRegression, LogisticRegression

from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

## Data Collection

In [ ]:
sp_cred = SpotifyClientCredentials(
    client_id=cid,
    client_secret=secret
)
sp = spotipy.Spotify(client_credentials_manager=sp_cred)

In [ ]:
all_good_ids = []
for good_playlist_id in good_playlist_ids:
  good_playlist = sp.user_playlist(username, good_playlist_id)
  good_tracks = good_playlist["tracks"]
  good_songs = good_tracks["items"]

  while good_tracks['next']:
      good_tracks = sp.next(good_tracks)
      for item in good_tracks["items"]:
          good_songs.append(item)

  good_ids = []
  for i in range(len(good_songs)):
      good_ids.append(good_songs[i]['track']['id'])
  print(len(good_ids))
  all_good_ids += good_ids
print("")
print(len(all_good_ids))
print(all_good_ids)

100
50
100
50
217

517
['2SAqBLGA283SUiwJ3xOUVI', '0mG6igzxbH55XRyq3ygEh8', '5IUtvfNvOyVYZUa6AJFrnP', '57VeLYXrvNxe8Vs18K2M9W', '18vXApRmJSgQ6wG2ll9AOg', '5sBplhIrmzA4kQGyIOpmDf', '0k7wmahjkn389wAZdz19Cv', '2MbdDtCv5LUVjYy9RuGTgC', '4TIqzdAssasqx3DAe6cG9J', '6ZoZ4KGIDD23DohdVk0Ybw', '6rTInqW3YECMkQsBEHw4sd', '33gwZOGJWEZ7dRWPqPxBEZ', '7o4gBbTM6UBLkOYPw9xMCz', '2hx8cZug1bgRMXgW5RCTUV', '1VLtjHwRWOVJiE5Py7JxoQ', '35RJhm1pEovTBwnNR0zWad', '1tkg4EHVoqnhR6iFEXb60y', '1JXBNvlDEl2epKSJ9EOgKX', '3w1WjD2zJqjBjDz5fwqQPJ', '3mDFLytDotXo2p0rvfGbkA', '0PJ4dXVkpNAtoOZsEjqktc', '26UxwWl9xCb83OynXELJcL', '1H7KnK26kc1YyellpbINEn', '2NM0qHzx4Ohy8iXHEEgwlO', '1inQjfrPjWSM8DehYMfIxi', '7v96VvFLS0wSYT8kewNLRy', '341lTIxmhybbQsgydwJiFx', '3zs87EV3EZSh2UpiXxT52Y', '527k23H0A4Q0UJN3vGs0Da', '21AJQhGZpujjZQXByZAXpr', '37Nqx7iavZpotJSDXZWbJ3', '0cu0rhnlCnJ7JIeHRiLpj3', '6wJYhPfqk3KGhHRG76WzOh', '2t7D3y0jvYjR8IP9Bosyew', '3McTpsUQfEDrlNd2ikro2E', '6EDO9iiTtwNv6waLwa1UUq', '3AMxuq6id3YGB57eWeheZQ', '2M2Pug4zwMial

In [ ]:
all_bad_ids = []
for bad_playlist_id in bad_playlist_ids:
  bad_playlist = sp.user_playlist(username, bad_playlist_id)
  bad_tracks = bad_playlist["tracks"]
  bad_songs = bad_tracks["items"]

  while bad_tracks['next']:
      bad_tracks = sp.next(bad_tracks)
      for item in bad_tracks["items"]:
          bad_songs.append(item)

  bad_ids = []
  for i in range(len(bad_songs)):
      bad_ids.append(bad_songs[i]['track']['id'])
  print(len(bad_ids))
  all_bad_ids += bad_ids
print("")
print(len(all_bad_ids))
print(all_bad_ids)

50
75
200
50
50
132

557
['0MKGH8UMfCnq5w7nG06oM5', '71SvEDmsOwIWw1IozsZoMA', '6KTv0Z8BmVqM7DPxbGzpVC', '2fuCquhmrzHpu5xcA1ci9x', '59WN2psjkt1tyaxjspN8fp', '7pKfPomDEeI4TPT6EOYjn9', '4pt5fDVTg5GhEvEtlz9dKk', '7N3PAbqfTjSEU1edb2tY8j', '003vvx7Niy0yvhvHt4a68B', '4BP3uh0hFLFRb5cjsgLqDh', '5zA8vzDGqPl2AzZkEYQGKh', '1f3yAtsJtY87CTmM8RLnxf', '1lCRw5FEZ1gPDNPzy1K4zW', '2zYzyRzz6pRmhPzyfMEC8s', '254bXAqt3zP6P50BdQvEsq', '5tVA6TkbaAH9QMITTQRrNv', '70LcF31zb1H0PyJoS1Sx1r', '1QFh8OH1e78dGd3VyJZCAC', '2Cdvbe2G4hZsnhNMKyGrie', '40riOy7x9W7GXjyGp4pjAv', '5T8EDUDqKcs6OSOwEsfqG7', '6Qyc6fS4DsZjB2mRW9DsQs', '6dGnYIeXmHdcikdzNNDMm2', '3gdewACMIVMEWVbyb8O9sY', '20OFwXhEXf12DzwXmaV7fj', '08mG3Y1vljYA6bvDt4Wqkj', '4o6BgsqLIBViaGVbx5rbRk', '4cIPLtg1avt2Jm3ne9S1zy', '11LmqTE2naFULdEP94AUBa', '2gT72HL4Y84K2Mle3miAMJ', '0ofHAoxe9vBkTCp2UQIavz', '5XcZRgJv3zMhTqCyESjQrF', '2RlgNHKcydI9sayD2Df2xp', '13CR9dPNbuH2li1xaepFe0', '40i81AcCCxiVEFUWElcjZC', '4bHsxqR3GMrXTxEPLuK5ue', '57bgtoPSgt236HzfBOd8kj', '37Tmv4NnfQe

In [ ]:
source_playlist = sp.user_playlist(username, source_playlist_id)
source_tracks = source_playlist["tracks"]
source_songs = source_tracks["items"]

while source_tracks['next']:
    source_tracks = sp.next(source_tracks)
    for item in source_tracks["items"]:
        source_songs.append(item)

source_ids = []
for i in range(len(source_songs)):
    if source_songs[i]['track'] != None:
      source_ids.append(source_songs[i]['track']['id'])
print(len(source_ids))
print(source_ids)

122
['0Dm43YLUlGdePpSbI1ct8h', '70eFcWOvlMObDhURTqT4Fv', '1DhRbox3xkceP64k3JeYfW', '3FHXUqaUer9o231Je1zxIN', '2md2i5QvelRFnafpnd6LOg', '1z3L82jeYnTmYUjPc7zXcu', '1MJ5f5EYBC92ADD6xcz7nb', '0W0R48uImgPVVJgb5YTC3I', '0nMqu5vN44GNv8UOQxmBJK', '5pgZpHqfv4TSomtkfGZGrG', '14rWEXQGBsFB7ALUc8G4sr', '4sqIBV3ukQDqRr8Me2nvQb', '1QvpsbjYAVVFvFrWVC0qnQ', '6Wz9rIfo9tOBcVCd1Mo7F7', '3OUyyDN7EZrL7i0Sbi5SVd', '32q9OuCp7wFdLZKuCXm0wl', '5JWPUEov2wlX7c0jhYZpeB', '2BITQ360Knh6qNAOqR7Dyq', '5O2Yjlcs5pSpHq1gcA0cwg', '23bOfmnZiN42pB1frXNyak', '1LwwZ0xoUz5gR8XrxEUKAK', '17aVK3oQdgwVG2l0HpP67W', '25nzKGDiua1lE9Qo5V19GL', '60dwJ8dnqZ4WrIdiu7LkNR', '7KrlNS2VqiNNJ82RMQdxBn', '2xdwCJLfKCkat14mlgR26M', '3Jbpo2slhW2bbVneiJnC9G', '4dVbhS6OiYvFikshyaQaCN', '5UWwZ5lm5PKu6eKsHAGxOk', '67vYdAAM3oGsEImCRfbtsF', '3ptB4SmuSJTGhQByFrMJVJ', '2CDVqsGGkf2v1tMETt3fq5', '4QMSoa7Fh3y4kmMNuBtOgt', '1F9kEuwT29fgqlovIclu81', '0EC0kuRKIUlru5eRxWYYyv', '6KYVWcOQWm7JrlsPMkrkNi', '732Sqj2piYPhdMPENP4axz', '0ooz46SAYMJsdOPnwW4nKK', '1mSClO

In [ ]:
training_features = []

for i in range(0, len(all_good_ids), 50):
  audio_features = sp.audio_features(all_good_ids[i : i + 50])
  for track in audio_features:
    if track != None:

      track['target'] = 1
      training_features.append(track)
      # training_features[-1]['target'] = 1

for i in range(0, len(all_bad_ids), 50):
  audio_features = sp.audio_features(all_bad_ids[i : i + 50])
  for track in audio_features:
    if track != None:
      track['target'] = 0
      training_features.append(track)
      # training_features[-1]['target'] = 0

print(len(training_features))
print(training_features)

1073
[{'danceability': 0.761, 'energy': 0.518, 'key': 0, 'loudness': -8.871, 'mode': 1, 'speechiness': 0.134, 'acousticness': 0.244, 'instrumentalness': 3.47e-05, 'liveness': 0.107, 'valence': 0.522, 'tempo': 133.976, 'type': 'audio_features', 'id': '2SAqBLGA283SUiwJ3xOUVI', 'uri': 'spotify:track:2SAqBLGA283SUiwJ3xOUVI', 'track_href': 'https://api.spotify.com/v1/tracks/2SAqBLGA283SUiwJ3xOUVI', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/2SAqBLGA283SUiwJ3xOUVI', 'duration_ms': 261493, 'time_signature': 4, 'target': 1}, {'danceability': 0.775, 'energy': 0.518, 'key': 6, 'loudness': -5.857, 'mode': 0, 'speechiness': 0.311, 'acousticness': 0.455, 'instrumentalness': 0, 'liveness': 0.121, 'valence': 0.646, 'tempo': 161.011, 'type': 'audio_features', 'id': '0mG6igzxbH55XRyq3ygEh8', 'uri': 'spotify:track:0mG6igzxbH55XRyq3ygEh8', 'track_href': 'https://api.spotify.com/v1/tracks/0mG6igzxbH55XRyq3ygEh8', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/0mG6igzxbH55XRyq3y

In [ ]:
source_features = []
for i in range(0, len(source_ids), 50):
    audio_features = sp.audio_features(source_ids[i : i + 50])
    for track in audio_features:
        track['target'] = 0
        source_features.append(track)
        # source_features[-1]['target'] = 0  # arbitrary

print(len(source_features))
print(source_features)

122
[{'danceability': 0.662, 'energy': 0.624, 'key': 10, 'loudness': -13.606, 'mode': 0, 'speechiness': 0.0417, 'acousticness': 0.165, 'instrumentalness': 0, 'liveness': 0.0674, 'valence': 0.961, 'tempo': 174.513, 'type': 'audio_features', 'id': '0Dm43YLUlGdePpSbI1ct8h', 'uri': 'spotify:track:0Dm43YLUlGdePpSbI1ct8h', 'track_href': 'https://api.spotify.com/v1/tracks/0Dm43YLUlGdePpSbI1ct8h', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/0Dm43YLUlGdePpSbI1ct8h', 'duration_ms': 252560, 'time_signature': 4, 'target': 0}, {'danceability': 0.64, 'energy': 0.647, 'key': 5, 'loudness': -8.114, 'mode': 0, 'speechiness': 0.186, 'acousticness': 0.124, 'instrumentalness': 0, 'liveness': 0.0801, 'valence': 0.546, 'tempo': 92.979, 'type': 'audio_features', 'id': '70eFcWOvlMObDhURTqT4Fv', 'uri': 'spotify:track:70eFcWOvlMObDhURTqT4Fv', 'track_href': 'https://api.spotify.com/v1/tracks/70eFcWOvlMObDhURTqT4Fv', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/70eFcWOvlMObDhURTqT4Fv'

## Data Preparation

In [ ]:
trainingData = pd.DataFrame(training_features)

In [ ]:
trainingData

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,target
0,0.761,0.518,0,-8.871,1,0.1340,0.24400,0.000035,0.107,0.522,133.976,audio_features,2SAqBLGA283SUiwJ3xOUVI,spotify:track:2SAqBLGA283SUiwJ3xOUVI,https://api.spotify.com/v1/tracks/2SAqBLGA283S...,https://api.spotify.com/v1/audio-analysis/2SAq...,261493,4,1
1,0.643,0.403,1,-11.254,0,0.4100,0.34300,0.000000,0.138,0.111,98.540,audio_features,1fxYz1s15pyZLsJT0Hdh1m,spotify:track:1fxYz1s15pyZLsJT0Hdh1m,https://api.spotify.com/v1/tracks/1fxYz1s15pyZ...,https://api.spotify.com/v1/audio-analysis/1fxY...,174682,4,1
2,0.829,0.512,6,-5.865,0,0.1870,0.49200,0.000000,0.120,0.638,80.511,audio_features,4XvcHTUfIlWfyJTRG0aqlo,spotify:track:4XvcHTUfIlWfyJTRG0aqlo,https://api.spotify.com/v1/tracks/4XvcHTUfIlWf...,https://api.spotify.com/v1/audio-analysis/4Xvc...,123987,4,1
3,0.904,0.723,11,-5.224,0,0.2600,0.06310,0.000000,0.185,0.835,145.013,audio_features,2MbdDtCv5LUVjYy9RuGTgC,spotify:track:2MbdDtCv5LUVjYy9RuGTgC,https://api.spotify.com/v1/tracks/2MbdDtCv5LUV...,https://api.spotify.com/v1/audio-analysis/2Mbd...,227478,4,1
4,0.782,0.510,4,-5.724,0,0.0419,0.14300,0.000000,0.115,0.118,99.993,audio_features,5IUtvfNvOyVYZUa6AJFrnP,spotify:track:5IUtvfNvOyVYZUa6AJFrnP,https://api.spotify.com/v1/tracks/5IUtvfNvOyVY...,https://api.spotify.com/v1/audio-analysis/5IUt...,143375,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1068,0.547,0.987,8,-3.146,1,0.0839,0.00191,0.006190,0.322,0.901,79.976,audio_features,6HjE9DWhhPLSlmIMEyZzRc,spotify:track:6HjE9DWhhPLSlmIMEyZzRc,https://api.spotify.com/v1/tracks/6HjE9DWhhPLS...,https://api.spotify.com/v1/audio-analysis/6HjE...,178627,4,0
1069,0.469,0.537,9,-10.421,1,0.1520,0.02410,0.002010,0.269,0.783,124.193,audio_features,3aoDEt6zSuYQ47gzarlaVo,spotify:track:3aoDEt6zSuYQ47gzarlaVo,https://api.spotify.com/v1/tracks/3aoDEt6zSuYQ...,https://api.spotify.com/v1/audio-analysis/3aoD...,382834,1,0
1070,0.322,0.265,2,-13.398,1,0.0302,0.07260,0.340000,0.366,0.207,133.577,audio_features,2gCf9PqIvpKbxwfJMBWwbn,spotify:track:2gCf9PqIvpKbxwfJMBWwbn,https://api.spotify.com/v1/tracks/2gCf9PqIvpKb...,https://api.spotify.com/v1/audio-analysis/2gCf...,226667,4,0
1071,0.373,0.453,9,-9.224,1,0.0419,0.26100,0.000046,0.160,0.418,95.384,audio_features,4nla8cAvm6AQFnNou9WiR1,spotify:track:4nla8cAvm6AQFnNou9WiR1,https://api.spotify.com/v1/tracks/4nla8cAvm6AQ...,https://api.spotify.com/v1/audio-analysis/4nla...,196841,4,0


In [ ]:
train, test = train_test_split(trainingData, test_size = 0.15)

In [ ]:
target_features = ["danceability", "loudness", "valence", "energy", "instrumentalness", "acousticness", "key", "speechiness", "duration_ms"]

x_train = train[target_features]
y_train = train["target"]
x_test = test[target_features]
y_test = test["target"]

In [ ]:
x_train

,danceability,loudness,valence,energy,instrumentalness,acousticness,key,speechiness,duration_ms
742,0.470,-12.824,0.243,0.144,0.000118,0.912000,10,0.0365,284560
282,0.630,-6.152,0.487,0.504,0.000000,0.095300,6,0.0334,249867
82,0.829,-4.937,0.312,0.343,0.000000,0.007340,2,0.1020,135431
401,0.769,-4.064,0.490,0.627,0.000000,0.394000,7,0.2620,164038
528,0.502,-4.556,0.720,0.912,0.000173,0.000025,1,0.0564,301920
...,...,...,...,...,...,...,...,...,...
117,0.766,-12.561,0.816,0.618,0.000050,0.032800,10,0.0790,184800
517,0.500,-5.846,0.539,0.824,0.001610,0.000160,4,0.0388,218920
42,0.719,-7.600,0.660,0.648,0.000000,0.200000,3,0.1250,123263
870,0.616,-5.968,0.386,0.438,0.000000,0.356000,1,0.0298,213387


In [ ]:
y_train

742    0
282    1
82     1
401    1
528    0
      ..
117    1
517    0
42     1
870    0
380    1
Name: target, Length: 912, dtype: int64

In [ ]:
x_test

,danceability,loudness,valence,energy,instrumentalness,acousticness,key,speechiness,duration_ms
400,0.570,-7.562,0.2700,0.703,0.000000,0.02000,6,0.2220,180016
263,0.793,-8.134,0.8980,0.465,0.000132,0.64800,5,0.2710,193740
752,0.597,-16.656,0.3400,0.128,0.000704,0.95900,9,0.0491,254400
536,0.579,-9.484,0.6090,0.508,0.000494,0.00574,2,0.0270,391376
569,0.521,-3.184,0.0868,0.831,0.000000,0.02900,1,0.0558,195000
...,...,...,...,...,...,...,...,...,...
925,0.685,-8.285,0.5780,0.521,0.000004,0.18500,2,0.0345,190779
996,0.336,-10.875,0.4270,0.792,0.001750,0.00351,9,0.0840,355304
329,0.813,-5.541,0.5470,0.808,0.000000,0.00693,2,0.2860,192132
725,0.421,-21.795,0.0546,0.080,0.859000,0.90300,7,0.0400,257693


In [ ]:
y_test

400    1
263    1
752    0
536    0
569    0
      ..
925    0
996    0
329    1
725    0
176    1
Name: target, Length: 161, dtype: int64

## Model Evaluation

### Random Forest

In [ ]:
# random forest combines multiple decision trees and uses bagging
# (training multiple trees on different sections of training data, averaging the result)
rfc = RandomForestClassifier(n_jobs=1, random_state=1)
rfc.fit(x_train, y_train)
rfc_pred = rfc.predict(x_test)

score = accuracy_score(y_test, rfc_pred) * 100
print("Accuracy using Random Forest: ", round(score, 2), "%")
print('Mean squared error: %.2f' % mean_squared_error(y_test, rfc_pred))

Accuracy using Random Forest:  88.82 %
Mean squared error: 0.11


Measured Accuracy: 88.82%

### Gradient Boosting

In [ ]:
# I <3 gradient boost
gbc = GradientBoostingClassifier(n_estimators=100, learning_rate=.1, max_depth=1, random_state=1)
gbc.fit(x_train, y_train)
gbc_pred = gbc.predict(x_test)

score = accuracy_score(y_test, gbc_pred) * 100
print("Accuracy using Gradient Boost: ", round(score, 2), "%")
print('Mean squared error: %.2f' % mean_squared_error(y_test, gbc_pred))

Accuracy using Gradient Boost:  90.68 %
Mean squared error: 0.09


Measured Accuracy: 90.68%

### Gradient Boosting w/ Manual Stacking (Random Forest)

In [ ]:
x_train_rfc = (x_train.iloc[0:int(len(x_train) / 2)]).copy()
y_train_rfc = (y_train.iloc[0:int(len(y_train) / 2)]).copy()
x_train_gbc = (x_train.iloc[int(len(x_train) / 2):len(x_train)]).copy()
y_train_gbc = (y_train.iloc[int(len(y_train) / 2):len(y_train)]).copy()

In [ ]:
x_test_rfc = x_train_gbc
y_test_rfc = y_train_gbc
x_test_gbc = x_test.copy()
y_test_gbc = y_test.copy()

In [ ]:
rfc_stack = RandomForestClassifier(n_jobs=1, random_state=1)
rfc_stack.fit(x_train_rfc, y_train_rfc)
rfc_stack_pred_train = rfc_stack.predict(x_test_rfc)

score = accuracy_score(y_test_rfc, rfc_stack_pred) * 100
print("Accuracy using Random Forest: ", round(score, 2), "%")
print('Mean squared error: %.2f' % mean_squared_error(y_test_rfc, rfc_stack_pred))

Accuracy using Random Forest:  87.72 %
Mean squared error: 0.12


In [ ]:
rfc_stack_pred_test = rfc_stack.predict(x_test_gbc)

In [ ]:
# stack
x_train_gbc['rfc_stack_pred'] = rfc_stack_pred_train.tolist()
x_test_gbc['rfc_stack_pred'] = rfc_stack_pred_test.tolist()

In [ ]:
x_train_gbc

,danceability,loudness,valence,energy,instrumentalness,acousticness,key,speechiness,duration_ms,rfc_stack_pred
1046,0.263,-6.189,0.504,0.922,0.001320,0.000029,9,0.0982,408853,0
57,0.751,-6.859,0.634,0.744,0.000568,0.152000,1,0.1770,171892,1
157,0.771,-6.844,0.182,0.526,0.000000,0.080400,1,0.0718,148927,1
179,0.697,-9.774,0.805,0.411,0.000000,0.407000,10,0.0891,166717,1
809,0.653,-11.567,0.249,0.368,0.003050,0.875000,10,0.0439,524093,0
...,...,...,...,...,...,...,...,...,...,...
117,0.766,-12.561,0.816,0.618,0.000050,0.032800,10,0.0790,184800,1
517,0.500,-5.846,0.539,0.824,0.001610,0.000160,4,0.0388,218920,0
42,0.719,-7.600,0.660,0.648,0.000000,0.200000,3,0.1250,123263,1
870,0.616,-5.968,0.386,0.438,0.000000,0.356000,1,0.0298,213387,0


In [ ]:
x_test_gbc

,danceability,loudness,valence,energy,instrumentalness,acousticness,key,speechiness,duration_ms,rfc_stack_pred
400,0.570,-7.562,0.2700,0.703,0.000000,0.02000,6,0.2220,180016,1
263,0.793,-8.134,0.8980,0.465,0.000132,0.64800,5,0.2710,193740,1
752,0.597,-16.656,0.3400,0.128,0.000704,0.95900,9,0.0491,254400,0
536,0.579,-9.484,0.6090,0.508,0.000494,0.00574,2,0.0270,391376,0
569,0.521,-3.184,0.0868,0.831,0.000000,0.02900,1,0.0558,195000,0
...,...,...,...,...,...,...,...,...,...,...
925,0.685,-8.285,0.5780,0.521,0.000004,0.18500,2,0.0345,190779,1
996,0.336,-10.875,0.4270,0.792,0.001750,0.00351,9,0.0840,355304,0
329,0.813,-5.541,0.5470,0.808,0.000000,0.00693,2,0.2860,192132,1
725,0.421,-21.795,0.0546,0.080,0.859000,0.90300,7,0.0400,257693,0


In [ ]:
gbc_stack = GradientBoostingClassifier(n_estimators=100, learning_rate=.1, max_depth=1, random_state=1)
gbc_stack.fit(x_train_gbc, y_train_gbc)
gbc_stack_pred = gbc_stack.predict(x_test_gbc)

score = accuracy_score(y_test_gbc, gbc_stack_pred) * 100
print("Accuracy using Gradient Boost: ", round(score, 2), "%")
print('Mean squared error: %.2f' % mean_squared_error(y_test_gbc, gbc_stack_pred))

Accuracy using Gradient Boost:  88.2 %
Mean squared error: 0.12


Measured Accuracy: 88.2%

### Stacking Classifier

In [ ]:
base_learners = [
  ('rf_1', RandomForestClassifier(n_estimators=10, random_state=42)),
  ('rf_2', KNeighborsClassifier(n_neighbors=5)),             
]

stk = StackingClassifier(estimators=base_learners, final_estimator=LogisticRegression())
stk.fit(x_train, y_train)
stk_pred = stk.predict(x_test)

score = accuracy_score(y_test, stk_pred) * 100
print("Accuracy using Gradient Boost: ", round(score, 2), "%")
print('Mean squared error: %.2f' % mean_squared_error(y_test, stk_pred))

Accuracy using Gradient Boost:  88.2 %
Mean squared error: 0.12


Measured Accuracy: 88.2%

In [ ]:
base_learners = [
  ('rf_1', RandomForestClassifier(n_estimators=10, random_state=42)),
  ('rf_2', KNeighborsClassifier(n_neighbors=5)),             
  ('rf_3', GradientBoostingClassifier(n_estimators=100, learning_rate=.1, max_depth=1, random_state=1))
]

stk = StackingClassifier(estimators=base_learners, final_estimator=LogisticRegression())
stk.fit(x_train, y_train)
stk_pred = stk.predict(x_test)

score = accuracy_score(y_test, stk_pred) * 100
print("Accuracy using Gradient Boost: ", round(score, 2), "%")
print('Mean squared error: %.2f' % mean_squared_error(y_test, stk_pred))

Accuracy using Gradient Boost:  89.44 %
Mean squared error: 0.11


Measured Accuracy: 89.44%

### Neural Networks

#### Baseline

In [ ]:
def keras_nn_baseline_model_small():
  model = Sequential()
  model.add(Dense(9, input_dim=9, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

In [ ]:
def keras_nn_baseline_model_big():
  model = Sequential()
  model.add(Dense(9, input_dim=9, activation='relu'))
  model.add(Dense(18, input_dim=9, activation='relu'))
  model.add(Dense(90, input_dim=18, activation='sigmoid'))
  model.add(Dense(1, activation='sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

In [ ]:
def keras_nn_baseline_model_shaped():
  model = Sequential()
  model.add(Dense(5, input_dim=9, activation='relu'))
  model.add(Dense(60, input_dim=5, activation='sigmoid'))
  model.add(Dense(5, input_dim=60, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

#### Pipeline w/ Normalization

In [ ]:
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=keras_nn_baseline_model_shaped, epochs=200, batch_size=5, verbose=0)))
keras_pipeline = Pipeline(estimators)

In [ ]:
kfold = StratifiedKFold(n_splits=10, shuffle=True)
results = cross_val_score(keras_pipeline, x_train, y_train, cv=kfold)
print("Standardized: %.2f%% (%.2f%%)" % (results.mean() * 100, results.std() * 100))

Standardized: 88.27% (2.06%)


**Cross-Validation Scores**  
keras_nn_baseline_model_small: 87.51%  
keras_nn_baseline_model_big: 85.09%  
keras_nn_baseline_model_shaped: 88.27%  

In [ ]:
keras_pipeline.fit(x_train, y_train)
keras_pred = keras_pipeline.predict(x_test)

score = accuracy_score(y_test, keras_pred) * 100
print("Accuracy using NN: ", round(score, 2), "%")
print('Mean squared error: %.2f' % mean_squared_error(y_test, keras_pred))

Accuracy using NN:  90.68 %
Mean squared error: 0.09


### Neural Networks w/ Stacking

In [ ]:
def get_nn_classifier():
  nn_clf = KerasClassifier(build_fn=keras_nn_baseline_model_shaped, epochs=200, batch_size=5, verbose=0)
  nn_clf._estimator_type = "classifier"
  estimators = []
  estimators.append(('standardize', StandardScaler()))
  estimators.append(('mlp', nn_clf))
  return Pipeline(estimators)

In [ ]:
base_learners = [
  ('rf_1', RandomForestClassifier(n_estimators=10, random_state=42)),
  ('rf_2', KNeighborsClassifier(n_neighbors=5)),             
  ('rf_3', GradientBoostingClassifier(n_estimators=100, learning_rate=.1, max_depth=1, random_state=1)),
  ('rf_4', get_nn_classifier())
]

nn_stk = StackingClassifier(estimators=base_learners, final_estimator=LogisticRegression())
nn_stk.fit(x_train, y_train)
nn_stk_pred = nn_stk.predict(x_test)

score = accuracy_score(y_test, nn_stk_pred) * 100
print("Accuracy using NN & Stacking: ", round(score, 2), "%")
print('Mean squared error: %.2f' % mean_squared_error(y_test, nn_stk_pred))

Accuracy using Gradient Boost:  90.06 %
Mean squared error: 0.10


Measured Accuracy: 90.06%

## Generate Predictions

In [ ]:
sourceData = pd.DataFrame(source_features)
sourceData = sourceData[target_features]

In [ ]:
sourceData

,danceability,loudness,valence,energy,instrumentalness,acousticness,key,speechiness,duration_ms
0,0.662,-13.606,0.9610,0.624,0.000000,0.16500,10,0.0417,252560
1,0.640,-8.114,0.5460,0.647,0.000000,0.12400,5,0.1860,197867
2,0.716,-5.313,0.9100,0.823,0.000000,0.13200,5,0.0495,161747
3,0.468,-6.536,0.7620,0.630,0.000000,0.31000,8,0.0843,204507
4,0.397,-4.577,0.5560,0.903,0.000002,0.00575,2,0.0457,201373
...,...,...,...,...,...,...,...,...,...
117,0.519,-9.982,0.4880,0.181,0.000000,0.94400,5,0.0795,82147
118,0.615,-8.615,0.7470,0.592,0.955000,0.04880,0,0.0443,263673
119,0.512,-5.128,0.8380,0.955,0.000013,0.17200,7,0.3540,40227
120,0.807,-9.729,0.9610,0.510,0.115000,0.04100,0,0.0433,279773


In [ ]:
# model = gbc
# model = keras_pipeline
model = nn_stk

In [ ]:
pred = model.predict(sourceData)

In [ ]:
source_playlist_songs[0]

{'added_at': '2020-10-01T05:24:42Z',
 'added_by': {'external_urls': {'spotify': 'https://open.spotify.com/user/agwx2'},
  'href': 'https://api.spotify.com/v1/users/agwx2',
  'id': 'agwx2',
  'type': 'user',
  'uri': 'spotify:user:agwx2'},
 'is_local': False,
 'primary_color': None,
 'track': None,
 'video_thumbnail': None}

In [ ]:

source_playlist_songs = source_playlist['tracks']['items']

print("ACCEPTED")


i = 0
for prediction in pred:
    if prediction == 1 and i + 1 < len(source_playlist_songs):
        song_title = source_playlist_songs[i + 1]['track']['name']
        artist_name = source_playlist_songs[i + 1]['track']['artists'][0]['name']
        print ("Song: " + song_title + ", By: "+ artist_name)
    i = i + 1


print("REJECTED")

i = 0
for prediction in pred:
    if prediction == 0 and i + 1 < len(source_playlist_songs):
        song_title = source_playlist_songs[i + 1]['track']['name']
        artist_name = source_playlist_songs[i + 1]['track']['artists'][0]['name']
        print ("Song: " + song_title + ", By: "+ artist_name)
    i = i + 1

ACCEPTED
Song: Beautiful People (feat. Khalid), By: Ed Sheeran
Song: Revolution 909, By: Daft Punk
Song: That's Life, By: Still Woozy
Song: Order More (feat. Starrah), By: G-Eazy
Song: Paper Trail$, By: Joey Bada$$
Song: Baby, By: Brandy
Song: I Like Dat, By: T-Pain
Song: Lost Cause, By: Billie Eilish
Song: STUNNAMAN (with Roddy Ricch & feat. Lil Wayne), By: Birdman
Song: Baddest (feat. Chris Brown & 2 Chainz), By: Yung Bleu
Song: Y U DON'T LOVE ME? (MISS AMERIKKKA), By: Joey Bada$$
Song: Easy Love, By: Lauv
Song: DRUGS, By: Tai Verdes
Song: Loser, By: Beck
Song: Run To The Sun, By: N.E.R.D
Song: Crush, By: Souly Had
Song: Didn't Cha Know, By: Erykah Badu
Song: Red (Brothel Remix), By: Lucy Daydream
Song: Rocky, By: Still Woozy
Song: Butterflies, By: Michael Jackson
Song: Bake Sale (feat. Travis Scott), By: Wiz Khalifa
Song: CASINO, By: Ashton Travis
Song: favorable colours, By: slenderbodies
Song: Outside, By: Weston Estate
Song: Another One Bites The Dust - Remastered 2011, By: Queen